In [1]:
import os
import torch
import model as models, trainer as trainers
from datamanager import dataset as all_datasets, DataManager
from main import resolve_trainer, resolve_optimizer, average_results, store_model, store_results
from collections import defaultdict


In [ ]:
!python main.py -m DAGMM -e 1 -d Arrhythmia --n-runs 1

In [1]:
import itertools as it
from bootstrap import available_datasets, available_models, train

available_datasets = ["Arrhythmia"] #list(set(available_datasets) - {"IDS2018", "USBIDS"})
available_models = ["MemAE"]
couples = it.product(list(set(available_models) - {"AE", "DUAD", "NeuTraLAD"}), available_datasets)
total = 0
dataset_root = "C:/Users/verdi/Documents/Datasets"
path_to_datasets_map = {
    "Arrhythmia": dataset_root + "/Arrhythmia/3_minified/arrhythmia_normalized.npz",
    "KDD10": dataset_root + "/KDD/3_minified/KDD10percent_minified.npz",
    "IDS2018": dataset_root + "/Thyroid/0_raw/thyroid.mat",
    "NSLKDD": dataset_root + "/NSL-KDD/3_minified/NSL-KDD_minified.npz",
    "USBIDS": None,
    "Thyroid": dataset_root + "/IDS2018/3_minified/feature_group_5_50percent.npz"
}
batch_sizes_map = {
    "Arrhythmia": 32,
    "KDD10": 1024,
    "IDS2018": 1024,
    "NSLKDD": 1024,
    "USBIDS": 1024,
    "Thyroid": 64
}
params = {
    "model_name": None,
    "dataset_name": None,
    "dataset_path": None,
    "batch_size": None,
    "pct": 1.,
    "corruption_ratio": 0.,
    "n_runs": 1,
    "n_epochs": 1,
    "learning_rate": 1e-4,
    "results_path": "../results",
    "models_path": "../models",
    "test_mode": False
}

for model_name, dataset_name in couples:
    print(model_name, dataset_name)
    params["model_name"] = model_name
    params["dataset_name"] = dataset_name
    params["dataset_path"] = path_to_datasets_map[dataset_name]
    params["batch_size"] = batch_sizes_map[dataset_name]
    train(**params)


MemAE Arrhythmia
Size of data with label =0 : 0.8539823008849557


  0%|          | 0/7 [00:00<?, ?it/s]

Run 1 of 1
Started training


 86%|████████▌ | 6/7 [00:00<00:00, 10.70it/s, epoch=1, loss=1.291]


Finished learning process
Evaluating model on test set


ValueError: not enough values to unpack (expected 3, got 2)

In [7]:
dataset_root = "C:/Users/verdi/Documents/Datasets/"
#datasets = ["KDD10", "NSLKDD", "Thyroid", "Arrhythmia", "IDS2018"]
datasets = ["IDS2018"]
datasets_classes = [
    # all_datasets.KDD10Dataset,
    # all_datasets.NSLKDDDataset,
    # all_datasets.ThyroidDataset,
    # all_datasets.ArrhythmiaDataset,
    all_datasets.IDS2018Dataset
]
path_to_datasets = [
    # dataset_root + "/KDD/3_minified/KDD10percent_minified.npz",
    # dataset_root + "/NSL-KDD/3_minified/NSL-KDD_minified.npz",
    # dataset_root + "/Thyroid/0_raw/thyroid.mat",
    # dataset_root + "/Arrhythmia/3_minified/arrhythmia_normalized.npz",
    dataset_root + "/IDS2018/3_minified/feature_group_5_50percent.npz"
]
#batch_sizes = (1024, 128, 64, 32, 1024)
batch_sizes = (1024,)

assert all([os.path.exists(p) for p in path_to_datasets])

NameError: name 'all_datasets' is not defined

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
n_epochs = 200
n_runs = 5

for ds_name, ds_cls, ds_path, batch_size in zip(datasets, datasets_classes, path_to_datasets, batch_sizes):
    print(f"Training on {ds_name}")
    ds_cls = ds_cls(ds_path)
    in_features = ds_cls.get_shape()[1]
    anomaly_ratio = 1 - ds_cls.anomaly_ratio
    all_results = defaultdict(list)

    best_f1 = 0.
    for r in range(n_runs):
        print(f"Run {r+1} of {n_runs}")
        model = models.MemAutoEncoder(ds_name, in_features, device=device)
        model_trainer = trainers.MemAETrainer(
            model=model,
            alpha=2e-4,
            lr=1e-4,
            n_epochs=n_epochs,
            batch_size=batch_size,
            device=device
        )
        train_set, test_set = ds_cls.train_test_split(test_ratio=0.50,)
        dm = DataManager(train_set, test_set, batch_size=batch_size)
        metrics = model_trainer.train(dm.get_train_set())
        print("Finished learning process")
        print("Evaluating model on test set")
        # We test with the minority samples as the positive class
        y_true, scores = model_trainer.test(dm.get_test_set())
        results = model_trainer.evaluate(y_true, scores, anomaly_ratio)
        print(results)
        for k, v in results.items():
            all_results[k].append(v)
        # Persist the best model
        if results.get("F1-Score") > best_f1:
            store_model(model, "DeepSVDD", ds_name)
        model.reset()
    final_results = average_results(all_results)
    params = dict(
                {"BatchSize": batch_size, "Epochs": n_epochs, "CorruptionRatio": 0., "Threshold": anomaly_ratio},
                **model.get_params()
            )
    fname = store_results(final_results, params, "MemAE", ds_name, ds_path)
    print(f"Results stored in {fname}")

Training on IDS2018
Run 1 of 5


  0%|          | 0/3270 [00:00<?, ?it/s]

Size of data with label =0 : 0.8297759557315137
Started training


100%|█████████▉| 3269/3270 [03:56<00:00, 13.82it/s, epoch=200, loss=0.398]


Finished learning process
Evaluating model on test set
{'Precision': 0.2932219691087702, 'Recall': 0.9997779342350417, 'F1-Score': 0.4534522451080901, 'AUROC': 0.6579017566947798, 'AUPR': 0.5660196892091182}
Run 2 of 5


...

Finished learning process
Evaluating model on test set
{'Precision': 0.29318248419642484, 'Recall': 0.9993956898855233, 'F1-Score': 0.45336571037793383, 'AUROC': 0.6651890474864829, 'AUPR': 0.5644826434698758}
Run 3 of 5
Size of data with label =0 : 0.8297759557315137


  0%|          | 0/3270 [00:00<?, ?it/s]

Started training


 78%|███████▊  | 2554/3270 [02:30<00:42, 16.97it/s, epoch=80, loss=0.466]


KeyboardInterrupt: 